In [15]:
import os 
path = "/home/mr/Grive/01.ALDA/Project-Alda/data/"
from pyspark.mllib.recommendation import ALS
import math

In [16]:
ratings = sc.textFile(os.path.join(path, 'ratings.csv')) \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line)>1) \
    .map(lambda a: (a[0], a[1], a[2]))
header = ratings.first()
ratings = ratings.filter(lambda line: line != header)
ratings = ratings.map(lambda x:(int(x[0]), x[1], x[2]))
print ("Total number of ratings : %d"%ratings.count())

Total number of ratings : 100004


In [17]:
movies = sc.textFile(os.path.join(path, 'movies.csv'))\
        .map(lambda line:line.split(","))\
        .filter(lambda line:len(line)>1)\
        .map(lambda a:(a[0],a[1]))
header2 = movies.first()
movies = movies.filter(lambda line: line != header2)
print ("Total number of Movies : %d"%movies.count())

Total number of Movies : 9125


Divide the data into Training data, Validation Data and Test data
Training data is to form the model with different rank everytime
For each time, sum of squared errors is found out using Validation data
Then the rank that gives least SSE is selected

In [18]:
seed = 133
trainData, validationData, testData = ratings.randomSplit([0.6, 0.2, 0.2], seed)
trainData.cache()
validationData.cache()
testData.cache()

PythonRDD[20] at RDD at PythonRDD.scala:43

Ranks chosen = [2,4,8,10]
ALS -> Alternating least squares that finds out latent factors 

In [19]:
rank_sse = {}
for rank in [2,4,8,10]:
    model = ALS.train(trainData, rank, seed=seed)
    predictions = model.predictAll(validationData.map(lambda x: (int(x[0]), int(x[1]))))\
                    .map(lambda r: ((r[0], r[1]), r[2]))
    actual_pred = validationData.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    rank_sse[rank] = actual_pred.map(lambda r: (r[1][0] - r[1][1])**2).sum()
print (rank_sse)

{8: 25635.63896171894, 2: 24326.618122816402, 4: 23567.3672340687, 10: 27560.814525813857}


lowest SSE for rank = 4
Construct model with rank = 4
Now SSE is calculated for the test data

In [20]:
model = ALS.train(trainData, rank=4, seed=seed)
predictions = model.predictAll(testData.map(lambda x: (int(x[0]), int(x[1])))).map(lambda r: ((r[0], r[1]), r[2]))
actual_pred = testData.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
sse = actual_pred.map(lambda r: (r[1][0] - r[1][1])**2).sum()  
print 'For testing data the SSE is %s' % (sse)

For testing data the SSE is 23075.5573174


Now a new user with some ratings is added to the ratings data
We predict his ratings to unwatched movies and sort based on predicted rating
and take the top 20 movies 

In [21]:
add_user = sc.parallelize([(0,260,4), (0,1,3), (0,16,3), (0,25,4), (0,32,4), (0,335,1), (0,379,1), (0,296,3),
     (0,858,5), (0,50,4)])
new_data = ratings.union(add_user)
model = ALS.train(new_data, rank=4, seed=133)
ids = add_user.map(lambda x:x[1]).collect()
to_predict = movies.filter(lambda x:int(x[0]) not in ids).map(lambda x:(0, int(x[0])))
predictions = model.predictAll(to_predict)
temp = movies.map(lambda x:(int(x[0]), x[1]))
predictions = predictions.map(lambda x: (x.rating, x.product)).sortByKey(ascending = False).map(lambda x:(x[1], x[0]))
recommend = predictions.take(5)
print (recommend)

[(670, 15.171682266858094), (4898, 13.150439981278964), (5490, 11.290043591976353), (63853, 10.138435535483822), (5477, 10.077309865367994)]


In [22]:
output = sc.parallelize(recommend)
output = output.join(temp).map(lambda x:x[1][1])
output.collect()

[u'The Big Bus (1976)',
 u'Australia (2008)',
 u'Novocaine (2001)',
 u'"World of Apu',
 u'Sex and Lucia (Luc\xeda y el sexo) (2001)']